In [1]:
# load packages 
library(pacman)
p_load(
    tidyverse, data.table, dtplyr, reshape2, 
    archive, kableExtra, SPARQL, janitor, 
    png, webp, Cairo, rsvg,
    httr, jsonlite)
# set option
options(dplyr.summarise.inform = FALSE)
gray_scale <- c('#F3F4F8','#D2D4DA',  '#bcc0ca', 
                '#D3D3D3', '#2B2B2B', 
                '#B3B5BD', '#838383', 
                '#9496A1', '#7d7f89', '#777986', 
                '#656673', '#5B5D6B', '#4d505e',
                '#404352', '#2b2d3b', '#282A3A',
                '#1b1c2a', '#191a2b',
                '#141626', '#101223')

In [2]:
# set up working directory
setwd('work/notebooks/patent')

In [62]:
# Code-Block 5. Extract patents for one-company full code
# read dataset 1, 2, and 3 
de_firms <- fread('./data/orbis_de_matched_l.csv')
han_names <- fread('./data/202208_HAN_NAMES.txt')
han_patents <- fread('./data/202208_HAN_PATENTS.txt')

# filter out germany firms from han_names 
# by setting Person_ctry_code == 'DE'
# match names "AIRBUS DEFENCE" and get their HAN_ID
airbus <- toupper('Airbus Defence')
han_names %>%
    .[Person_ctry_code == 'DE'] %>%
    .[Clean_name %like% airbus] %>%
    .[,HAN_ID] -> airbus_han_ids

# calcualte the summary statistics for AIRBUS DEFENCE
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[, .N, by=Publn_auth] -> foo
    transform(adorn_totals(foo)) %>%
    transpose() %>%
    row_to_names(row_number=1)

# focusing on patents from EPO
# filter with condition Publn_auth == 'EP'
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] -> airbus_han_patents

# save the dataset
write.csv(airbus_han_patents, './data/airbus_han_patents.csv', row.names=FALSE)

,EP,US,WO,Total
,<chr>,<chr>,<chr>,<chr>
2,716,415,88,1219


In [3]:
airbus_ep_application <- fread('./data/airbus_ep_applications.csv')

In [4]:
dim(airbus_ep_application)

[1] 716   7

In [5]:
head(airbus_ep_application)

patentNumber,applicationDate,granted,grantDate,familyID,cpcTags,publicationItems
<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>
EP2030891,"Wed, 04 Jun 2008",0,,39745168,B64C 1/068,"['EP 2030891 A2', 'EP 2030891 A3']"
EP2025928,"Thu, 24 Jul 2008",0,,40039666,"['F03D 1/0675', 'F03D 3/062']","['EP 2025928 A2', 'EP 2025928 A3']"
EP1920908,"Fri, 02 Nov 2007",1,"Wed, 08 Apr 2015",39118022,"['B29C 70/386', 'B29C 70/541', 'B29C 70/543', 'B29C 70/545', 'B29C 70/56']","['EP 1920908 A1', 'EP 1920908 B1']"
EP1972896,"Sat, 08 Mar 2008",1,"Wed, 06 May 2015",39544979,G01C 23/00,"['EP 1972896 A2', 'EP 1972896 A3', 'EP 1972896 B1']"
EP2134522,"Thu, 13 Mar 2008",1,"Wed, 01 Mar 2017",39595708,"['B29B 11/16', 'B29C 70/382', 'B29C 70/386', 'B29C 70/545', 'B65H 49/18', 'B65H 51/005', 'B65H 57/16', 'D02J 1/18', 'D04H 1/4242', 'D04H 1/60', 'D04H 1/655', 'D04H 13/00', 'D04H 3/002', 'D04H 3/04', 'D04H 3/12']","['EP 2134522 A1', 'EP 2134522 B1', 'EP 2134522 B8']"
EP2136979,"Thu, 13 Mar 2008",1,"Wed, 11 May 2016",39495747,"['B29B 11/16', 'B29C 31/085', 'B29C 70/386', 'B29C 70/541', 'B29C 70/543', 'B29C 70/545']","['EP 2136979 A1', 'EP 2136979 B1']"


In [6]:
airbus_ep_publications <- fread('./data/airbus_ep_publications.csv')

In [7]:
dim(airbus_ep_publications)

[1] 538   5

In [8]:
head(airbus_ep_publications)

patentNumber,publicationDate,priorityNumber,language,ipc
<chr>,<chr>,<chr>,<chr>,<chr>
EP1920908,"Wed, 08 Apr 2015",102006052592,de,"['B29C 70/38', 'B29C 70/54', 'B29C 70/56']"
EP1972896,"Wed, 06 May 2015",102007014015,de,"['B64D 45/08', 'G01C 23/00']"
EP2134522,"Wed, 01 Mar 2017",102007012608,de,"['B29B 11/16', 'B29C 70/20', 'B29C 70/38', 'B29C 70/54', 'B65H 49/18', 'B65H 51/005', 'B65H 57/16', 'D02J 1/18', 'D04H 1/4242', 'D04H 1/60', 'D04H 1/655', 'D04H 13/00', 'D04H 3/002', 'D04H 3/04', 'D04H 3/12']"
EP2136979,"Wed, 11 May 2016",102007012609,de,"['B29B 11/16', 'B29C 70/38', 'B29C 70/54']"
EP1976108,"Wed, 21 Oct 2020",102007015302,de,"['H02M 3/337', 'H02M 7/5387']"
EP2039604,"Wed, 01 Mar 2017",102007045205,de,"['B64D 1/04', 'F41A 9/09', 'G01B 7/00', 'G01B 7/14', 'G01S 5/16']"
